# Milestone 1: Data Pipeline Exploration

This notebook provides interactive exploration of the multi-source data pipeline:
- **PriceLoader**: Yahoo Finance OHLCV + technical features
- **FREDLoader**: Macroeconomic indicators from FRED
- **EDGARLoader**: SEC filings (10-K, 10-Q)
- **DataPipeline**: Unified data alignment

Run each cell to explore the data and verify everything works correctly.

## Setup

In [ ]:
import sys
from pathlib import Path

# Add src to path
sys.path.insert(0, str(Path.cwd().parent / 'src'))

# Load environment variables
from dotenv import load_dotenv
load_dotenv(Path.cwd().parent / '.env')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configure display
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 200)
plt.style.use('seaborn-v0_8-whitegrid')

print("Setup complete!")

---
## 1. Price Data (Yahoo Finance)

Load OHLCV data and explore technical features.

In [ ]:
from data.price_loader import PriceLoader

price_loader = PriceLoader()

# Fetch SPY data
price_df = price_loader.get_price_data(
    symbol='SPY',
    start_date='2020-01-01',
    end_date='2024-01-01'
)

print(f"Shape: {price_df.shape}")
print(f"Date range: {price_df.index.min()} to {price_df.index.max()}")
print(f"\nColumns: {list(price_df.columns)}")
price_df.tail()

In [ ]:
# Add technical features
price_features = price_loader.add_technical_features(price_df)

print(f"Total features: {len(price_features.columns)}")
print(f"\nNew columns added:")
new_cols = [c for c in price_features.columns if c not in price_df.columns]
for col in new_cols:
    print(f"  - {col}")

In [ ]:
# Visualize price and some indicators
fig, axes = plt.subplots(3, 1, figsize=(14, 10), sharex=True)

# Price with SMAs
ax1 = axes[0]
ax1.plot(price_features.index, price_features['Close'], label='Close', alpha=0.8)
ax1.plot(price_features.index, price_features['sma_50'], label='SMA 50', alpha=0.7)
ax1.plot(price_features.index, price_features['sma_200'], label='SMA 200', alpha=0.7)
ax1.set_ylabel('Price ($)')
ax1.legend()
ax1.set_title('SPY Price with Moving Averages')

# RSI
ax2 = axes[1]
ax2.plot(price_features.index, price_features['rsi_14'], color='purple')
ax2.axhline(70, color='red', linestyle='--', alpha=0.5, label='Overbought')
ax2.axhline(30, color='green', linestyle='--', alpha=0.5, label='Oversold')
ax2.set_ylabel('RSI')
ax2.set_ylim(0, 100)
ax2.legend()
ax2.set_title('RSI (14-day)')

# Volatility
ax3 = axes[2]
ax3.plot(price_features.index, price_features['volatility_21d'] * 100, label='21-day Vol', color='orange')
ax3.set_ylabel('Volatility (%)')
ax3.set_xlabel('Date')
ax3.legend()
ax3.set_title('Annualized Volatility')

plt.tight_layout()
plt.show()

---
## 2. Macro Data (FRED)

Load macroeconomic indicators used for regime detection.

In [ ]:
from data.fred_loader import FREDLoader, FRED_SERIES, DEFAULT_REGIME_SERIES

# Show available series
print("Available FRED Series:")
for series_id, description in FRED_SERIES.items():
    marker = "*" if series_id in DEFAULT_REGIME_SERIES else " "
    print(f"  {marker} {series_id}: {description}")

print(f"\n* = Default regime detection series")

In [ ]:
fred_loader = FREDLoader()

# Fetch macro indicators
macro_df = fred_loader.get_macro_indicators(
    series=DEFAULT_REGIME_SERIES,
    start_date='2020-01-01',
    end_date='2024-01-01'
)

print(f"Shape: {macro_df.shape}")
print(f"Date range: {macro_df.index.min()} to {macro_df.index.max()}")
macro_df.tail(10)

In [ ]:
# Visualize key macro indicators
fig, axes = plt.subplots(3, 2, figsize=(14, 10))

# Yield Curve (10Y-2Y spread)
ax = axes[0, 0]
ax.plot(macro_df.index, macro_df['T10Y2Y'], color='blue')
ax.axhline(0, color='red', linestyle='--', alpha=0.7, label='Inversion threshold')
ax.fill_between(macro_df.index, macro_df['T10Y2Y'], 0, 
                where=macro_df['T10Y2Y'] < 0, color='red', alpha=0.3, label='Inverted')
ax.set_ylabel('Spread (%)')
ax.set_title('Yield Curve (10Y - 2Y Treasury)')
ax.legend()

# VIX
ax = axes[0, 1]
ax.plot(macro_df.index, macro_df['VIXCLS'], color='purple')
ax.axhline(20, color='green', linestyle='--', alpha=0.5, label='Low vol')
ax.axhline(30, color='orange', linestyle='--', alpha=0.5, label='Elevated')
ax.set_ylabel('VIX')
ax.set_title('VIX Volatility Index')
ax.legend()

# Unemployment
ax = axes[1, 0]
ax.plot(macro_df.index, macro_df['UNRATE'], color='brown')
ax.set_ylabel('Rate (%)')
ax.set_title('Unemployment Rate')

# Fed Funds Rate
ax = axes[1, 1]
ax.plot(macro_df.index, macro_df['FEDFUNDS'], color='green')
ax.set_ylabel('Rate (%)')
ax.set_title('Federal Funds Rate')

# Credit Spread
ax = axes[2, 0]
ax.plot(macro_df.index, macro_df['BAA10Y'], color='red')
ax.axhline(2, color='gray', linestyle='--', alpha=0.5)
ax.set_ylabel('Spread (%)')
ax.set_title('BAA Corporate Bond Spread')
ax.set_xlabel('Date')

# Correlation heatmap
ax = axes[2, 1]
corr = macro_df.corr()
sns.heatmap(corr, annot=True, cmap='RdBu_r', center=0, ax=ax, fmt='.2f')
ax.set_title('Correlation Matrix')

plt.tight_layout()
plt.show()

In [ ]:
# Add derived features
macro_features = fred_loader.calculate_derived_features(macro_df)

print("Derived features added:")
new_cols = [c for c in macro_features.columns if c not in macro_df.columns]
for col in new_cols:
    print(f"  - {col}")

# Show yield curve inversion periods
inverted_periods = macro_features[macro_features['T10Y2Y_inverted'] == 1]
print(f"\nYield curve inverted on {len(inverted_periods)} days")
print(f"Inversion periods: {inverted_periods.index.min()} to {inverted_periods.index.max()}")

---
## 3. SEC Filings (EDGAR)

Explore SEC filing data for a sample company.

In [ ]:
from data.edgar_loader import EDGARLoader

edgar_loader = EDGARLoader(email="research@example.com")

# Get CIK for Apple
cik = edgar_loader.get_cik('AAPL')
print(f"Apple CIK: {cik}")

In [ ]:
# Fetch recent 10-K filings
filings = edgar_loader.get_filings(
    ticker='AAPL',
    filing_type='10-K',
    count=5
)

print(f"Found {len(filings)} 10-K filings:\n")
for f in filings:
    print(f"  {f.filing_date}: {f.filing_type}")
    print(f"    Company: {f.company_name}")
    print(f"    URL: {f.document_url[:80]}...\n")

In [ ]:
# Get filings as DataFrame (without extracting text for speed)
filings_df = edgar_loader.get_filings_dataframe(
    'AAPL',
    filing_types=['10-K', '10-Q'],
    start_date='2020-01-01',
    extract_text=False
)

print(f"Total filings: {len(filings_df)}")
print(f"\nFiling type distribution:")
print(filings_df['filing_type'].value_counts())

filings_df[['filing_date', 'filing_type', 'company_name']].head(10)

In [ ]:
# Optional: Extract text from most recent filing (takes a few seconds)
# Uncomment to run

# if filings:
#     print(f"Extracting text from {filings[0].filing_date} 10-K...")
#     text_sections = edgar_loader.extract_filing_text(filings[0])
#     
#     for section, text in text_sections.items():
#         print(f"\n{section}: {len(text):,} characters")
#         print(f"Preview: {text[:500]}...")

---
## 4. Unified Data Pipeline

Load and align data from all sources.

In [ ]:
from data.data_pipeline import DataPipeline

pipeline = DataPipeline()

# Load aligned data (price + macro)
aligned_df = pipeline.load_aligned_data(
    symbol='SPY',
    start_date='2020-01-01',
    end_date='2024-01-01',
    include_price=True,
    include_macro=True,
    include_filings=False  # Skip for speed
)

print(f"Aligned data shape: {aligned_df.shape}")
print(f"Date range: {aligned_df.index.min()} to {aligned_df.index.max()}")

# Group columns by source
price_cols = [c for c in aligned_df.columns if c.startswith('price_')]
macro_cols = [c for c in aligned_df.columns if c.startswith('macro_')]

print(f"\nPrice features: {len(price_cols)}")
print(f"Macro features: {len(macro_cols)}")

In [ ]:
# Check for missing values
missing = aligned_df.isna().sum()
missing_pct = (missing / len(aligned_df) * 100).round(2)

print("Missing values (>0%):")
print(missing_pct[missing_pct > 0].sort_values(ascending=False).head(20))

In [ ]:
# Create modeling dataset
X, y = pipeline.create_modeling_dataset(
    symbol='SPY',
    start_date='2020-01-01',
    end_date='2024-01-01',
    target_horizon=1,
    target_type='direction'
)

print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")
print(f"\nTarget distribution:")
print(y.value_counts(normalize=True).round(3))

In [ ]:
# Create train/val/test split
splits = pipeline.get_train_test_split(X, y, test_ratio=0.2, val_ratio=0.1)

print("Data Splits (Time-Based):")
print("=" * 50)
for name, (X_split, y_split) in splits.items():
    print(f"\n{name.upper()}:")
    print(f"  Samples: {len(X_split):,}")
    print(f"  Date range: {X_split.index.min().date()} to {X_split.index.max().date()}")
    print(f"  Target balance: {y_split.mean():.2%} positive")

In [ ]:
# Visualize train/val/test split
fig, ax = plt.subplots(figsize=(14, 4))

colors = {'train': 'blue', 'val': 'orange', 'test': 'green'}
for name, (X_split, y_split) in splits.items():
    ax.axvspan(X_split.index.min(), X_split.index.max(), 
               alpha=0.3, color=colors[name], label=f"{name} ({len(X_split)} samples)")

# Overlay price
ax2 = ax.twinx()
ax2.plot(aligned_df.index, aligned_df['price_Close'], color='black', alpha=0.5, linewidth=0.5)
ax2.set_ylabel('SPY Price')

ax.set_xlabel('Date')
ax.set_title('Time-Based Train/Validation/Test Split')
ax.legend(loc='upper left')

plt.tight_layout()
plt.show()

---
## 5. Feature Analysis

Explore feature distributions and correlations.

In [ ]:
# Feature statistics
X_train, y_train = splits['train']

print("Feature Statistics (Training Set):")
X_train.describe().T.head(20)

In [ ]:
# Correlation with target (on training set only)
correlations = X_train.corrwith(y_train).sort_values(key=abs, ascending=False)

print("Top 15 features correlated with target:")
print(correlations.head(15).round(4))

print("\nBottom 15 features (least correlated):")
print(correlations.tail(15).round(4))

In [ ]:
# Visualize top correlations
top_features = correlations.head(10).index.tolist()

fig, axes = plt.subplots(2, 5, figsize=(16, 6))
axes = axes.flatten()

for i, feature in enumerate(top_features):
    ax = axes[i]
    
    # Box plot by target class
    for label in [0, 1]:
        data = X_train.loc[y_train == label, feature].dropna()
        color = 'red' if label == 0 else 'green'
        ax.hist(data, bins=30, alpha=0.5, color=color, label=f"{'Down' if label == 0 else 'Up'}")
    
    ax.set_title(f"{feature}\n(corr={correlations[feature]:.3f})")
    ax.legend(fontsize=8)

plt.suptitle('Feature Distributions by Target Class (Training Set)', y=1.02)
plt.tight_layout()
plt.show()

---
## Summary

The data pipeline is working correctly:

1. **PriceLoader**: Successfully fetches OHLCV and computes 20+ technical features
2. **FREDLoader**: Fetches 5 key macro indicators with derived features
3. **EDGARLoader**: Can fetch and parse SEC filings
4. **DataPipeline**: Aligns all sources to daily frequency with proper time-based splits

**Next Steps**: Milestone 2 - Regime Detection using the macro indicators.